**Submission Notebook**

This notebook serves as a guide to run inference on the model.

**Installing and Importing Packages**

In [ ]:
!pip install patchify
!pip install cv2

import os
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
from PIL import Image
import zipfile
from skimage.measure import block_reduce
from patchify import patchify
from patchify import unpatchify
from xml.dom.minidom import parse
import re
from PIL import Image
from google.colab.patches import cv2_imsho

**Defining Useful Variables**

In [ ]:
INPUT_DIR = os.path.join(os.getcwd(), 'tmc_input')
print(INPUT_DIR)
filename = "tmc_ch2"
PATCH_SIZE = 16
DOWNSAMPLE_RES = 4
SAMPLE_METHOD = "bicubic"
num_im = 1

**Defining Functions**

In [ ]:
def get_shape(xml_filename):
    dom = parse(xml_filename)
    name = dom.getElementsByTagName('elements')
    print (name[0].firstChild.nodeValue)
    print (name[1].firstChild.nodeValue)
    return (int(name[0].firstChild.nodeValue), int(name[1].firstChild.nodeValue))

def check2(im):
    k=0
    for i in range(1, 64):
        for j in range(1, 64):
            if im[i][j]==255:
                k+=1
    return k

def get_patches(image):
    patches = patchify(image, (PATCH_SIZE,PATCH_SIZE), step=PATCH_SIZE)
    print(patches.shape)
    return patches

def isValid(img, x=0.9):
    return (np.corrcoef(cv.medianBlur(img, 3).ravel(), img.ravel())[0][1] > x)

def is_monochromatic_image(img):
    extr = Image.fromarray(img).getextrema()
    a = 0
    for i in extr:
        if isinstance(i, tuple):
            a += abs(i[0] - i[1])
        else:
            a = abs(extr[0] - extr[1])
            break
    return a == 0

def save(patches):
    num_im=0
    path = INPUT_DIR + "patches"
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)
    for i in range(patches.shape[0]):
        for j in range(3, patches.shape[1]):
            if not(is_monochromatic_image(patches[i][j])):
                plt.imsave(path + f"{i}_{j}.png", patches[i][j]*255, cmap='gray')
                num_im += 1
    print('Done')

def get_noises(patches):
    patch_noise = []

    for i in range(patches.shape[0]):
        patch_noise.append([])
        for j in range(patches.shape[1]):
            patch_noise[-1].append(np.corrcoef(cv.medianBlur(patches[i][j], 3).ravel(),patches[i][j].ravel())[0][1])
    return patch_noise

def show_plots(patches):
    noises = get_noises(patches)
    noises = np.array(noises)

    plt.subplot(2, 1, 1)
    plt.hist(noises)
    plt.title('Patch Noises')

    plt.subplot(2, 1, 2)
    plt.imshow(noises,  cmap='hot', interpolation='nearest')

    plt.show()

In [ ]:
input_image = filename + '.tif'
image_path = os.path.join(INPUT_DIR, input_image)
print(image_path)
img = cv.imread(input_image, -1)
image = np.asarray(img)
print(img.shape)
patches = get_patches(image)
save(patches)
plt.imshow(img, cmap = "gray")
plt.show()            

**Creating High Resolution Images Out of the Patches**

Writing on some files

In [ ]:
%%writefile /content/HAT/options/test/HAT-L_SRx16_config.yml

name: HAT-L_SRx16
model_type: HATModel
scale: 16
num_gpu: 1  # set num_gpu: 0 for cpu mode
manual_seed: 0

datasets:
  test_1:  # the 1st test dataset
    name: ISRO
    type: PairedImageDataset
    dataroot_gt: /content/drive/MyDrive/Test_Patches # PUT HR IMGS FOLDER PATH HERE
    dataroot_lq: /content/drive/MyDrive/Test_Patches # PUT LR IMGS FOLDER PATH HERE
    io_backend:
      type: disk

  # test_2:  # the 2nd test dataset
  #   name: Set14
  #   type: PairedImageDataset
  #   dataroot_gt: ./datasets/Set14/GTmod4
  #   dataroot_lq: ./datasets/Set14/LRbicx4
  #   io_backend:
  #     type: disk

  # test_3:
  #   name: Urban100
  #   type: PairedImageDataset
  #   dataroot_gt: ./datasets/urban100/GTmod4
  #   dataroot_lq: ./datasets/urban100/LRbicx4
  #   io_backend:
  #     type: disk

  # test_4:
  #    name: BSDS100
  #    type: PairedImageDataset
  #    dataroot_gt: ./datasets/BSDS100/GTmod4
  #    dataroot_lq: ./datasets/BSDS100/LRbicx4
  #    io_backend:
  #      type: disk

  # test_5:
  #     name: Manga109
  #     type: PairedImageDataset
  #     dataroot_gt: ./datasets/manga109/GTmod4
  #     dataroot_lq: ./datasets/manga109/LRbicx4
  #     io_backend:
  #       type: disk

# network structures
network_g:
  type: HAT
  upscale: 16
  in_chans: 3
  img_size: 16
  window_size: 4
  compress_ratio: 3
  squeeze_factor: 30
  conv_scale: 0.01
  overlap_ratio: 0.5
  img_range: 1.
  depths: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
  embed_dim: 180
  num_heads: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
  mlp_ratio: 2
  upsampler: 'pixelshuffle'
  resi_connection: '1conv'


# path
path:
  pretrain_network_g: # PUT MODEL PATH HERE
  strict_load_g: true
  param_key_g: 'params_ema'

# validation settings
val:
  save_img: true
  suffix: ~  # add suffix to saved images, if None, use exp name

  metrics:
    psnr: # metric name, can be arbitrary
      type: calculate_psnr
      crop_border: 4
      test_y_channel: true
    ssim:
      type: calculate_ssim
      crop_border: 4
      test_y_channel: true


Testing

In [ ]:
!python /content/HAT/hat/test.py -opt /content/HAT/options/test/HAT-L_SRx16_config.yml

**Joining the Patches Back**

In [ ]:
path = os.path.join(os.getcwd(), 'HAT/results/HAT-L_SRx16/visualization/ISRO/')
print(path)
rec_arr = np.empty((int(patches.shape[0]), int(patches.shape[1]), PATCH_SIZE, PATCH_SIZE))
for file_name in os.listdir(path):
    file = path + file_name
    if os.path.isfile(file):
        temp = re.findall(r'\d+', file_name)
        res = list(map(int, temp))
        rec_arr[int(res[0])][int(res[1])] = cv.imread(file)[:,:,0]

In [ ]:
rec_im = unpatchify(patches, (patches.shape[0]*PATCH_SIZE, patches.shape[1]*PATCH_SIZE))
plt.imshow(rec_im, cmap = "gray")
plt.show()
output_path = os.path.join(os.getcwd(), 'upscaled_output')
print(output_path)
plt.imsave('/content/gdrive/MyDrive/joined.png', rec_im, cmap='gray')

**Converting the upsampled .png to .tif file**

In [ ]:
import gdal
import subprocess
from subprocess import PIPE
from bs4 import BeautifulSoup

In [ ]:
XML_PATH = INPUT_PATH + INPUT
OUTPUT_IMG_NAME = "output"

In [ ]:
with open(f"{XML_PATH+'.xml'}", 'r') as f:
    data = f.read()
Bs_data = BeautifulSoup(data, "xml")
data = Bs_data.find_all('Axis_Array')
for item in data:
    if item.axis_name.string == 'Line':
        height = int(item.elements.string)*16
    if item.axis_name.string == 'Sample':
        width = int(item.elements.string)*16
data = Bs_data.find('isda:Corrected_Corner_Coordinates').find_all()
lats, longs = [], []
for index, item in enumerate(data):
    if index%2:
        longs.append(float(item.string))
    else:
        lats.append(float(item.string))

In [ ]:
result = subprocess.run(f"gdal_translate -ot Int16 -of GTiff -a_srs .\wkt.prj -gcp 0 0 {longs[0]} {lats[0]} -gcp {width} 0 {longs[1]} {lats[1]} -gcp 0 {height} {longs[2]} {lats[2]} -gcp {width} {height} {longs[3]} {lats[3]} {input_img_name+'.png'} {output_img_name+'.tif'}", stdout=PIPE, stderr=PIPE)

In [ ]:
print(result.stdout)